In [1]:
from Bio import SeqIO
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
'''
nextclade run \
   --input-dataset data/sars-cov-2 \
   --output-all=/home/soniali/Desktop/02_china_recom_renew/2_treetime/nextclade_output \
   /home/soniali/Desktop/02_china_recom_renew/2_treetime/covsampler_3945/Qualified-china-comprehensive-subsample-3945.fasta
'''
'''
cd /home/soniali/Desktop/02_china_recom_renew/2_treetime
'''
'''
FastTree -gtr -nt ./nextclade_output/nextclade.aligned.fasta > aln_3945_tree.nwk
'''
# '''  
# treetime ancestral --aln ./nextclade_output/nextclade.aligned.fasta --tree aln_3945_tree.nwk
# '''
'''
treetime \
    --aln ./nextclade_output/nextclade.aligned.fasta \
    --tree aln_3945_tree.nwk \
    --dates Qualified_china_meta-comp-3945.tsv \
    --reroot least-squares
    --confidence
    
'''
## 无需要删除的tips，舍弃下方注释步骤
	# '''
	# Rscript scripts/drop_tips_from_phylo_tree.R
	# '''

# '''

'''
treetime mugration \
    --tree ./2024-02-23_treetime/timetree.nexus \
    --states Qualified_china_meta-comp-3945.tsv \
    --attribute province \
    --confidence 
'''

In [11]:
# calculate numerical date

import datetime

def date_to_numerical(date_str):
    date = datetime.datetime.strptime(date_str, '%Y-%m-%d')
    year_start = datetime.datetime(date.year, 1, 1)
    days_since_start = (date - year_start).days + 1
    fraction_of_year = days_since_start / (366 if date.year % 4 == 0 and (date.year % 100 != 0 or date.year % 400 == 0) else 365)
    numerical_date = date.year + fraction_of_year
    return round(numerical_date, 6)

date_to_numerical('2023-10-31')

2023.832877

In [12]:
import baltic as bt
import pandas as pd

treeFile = "/home/soniali/Desktop/02_china_recom_renew/2_treetime/2024-02-23_mugration/annotated_tree.nexus"
outFile = '/home/soniali/Desktop/02_china_recom_renew/2_treetime/2024-02-23_mugration/annotated_tree_events.csv'
myTree=bt.loadNewick(treeFile, absoluteTime=False)
myTree.setAbsoluteTime(2023.832877) # need to set this to time of last sampled tip which for our data set is 2023-10-31

myTree.traverse_tree() ## required to set heights
myTree.treeStats() ## report stats about tree

changes = 0
times = []
origins = []
destinations = []
for k in myTree.Objects: ## iterate over a flat list of branches
    # print([k.absoluteTime])
    "Assign node UNKNOWN region if not give"
    if 'province' in k.traits: # change to region or new_region if needed
        region = k.traits['province'] # change to region or new_region if needed
    else:
        region = 'UNKNOWN'
        k.traits['province'] = region # change to region or new_region if needed
    
    "Find parent region if given"
    if k.parent.traits:
        parent_region = k.parent.traits['province'] # change to region or new_region if needed
    else:
        parent_region = 'UNKNOWN'
        
    if region != parent_region:
        changes += 1
        times = times + [k.absoluteTime]
        origins = origins + [parent_region]
        destinations = destinations + [region]
        
        
print("Total number of state changes: " + str(changes))

df = pd.DataFrame({'EventTime' : times})
df['Origin'] = origins
df['Destination'] = destinations

df.to_csv(outFile)


Tree height: 3.365190
Tree length: 908.662530
annotations present

Numbers of objects in tree: 6567 (2622 nodes and 3945 leaves)

Total number of state changes: 2101


In [13]:
k.absoluteTime

2023.780817

In [ ]:
# "annotated_tree_events_del.csv"是删除了"annotated_tree_events.csv"中的含有"UNKNOWN"行后的文件

In [2]:
dirpath = "/home/soniali/Desktop/02_china_recom_renew/2_treetime/2024-02-23_mugration/"
df = pd.read_csv(dirpath+"annotated_tree_events_del.csv")
df

,Unnamed: 0,EventTime,Origin,Destination
0,1103,2022.193397,Hong Kong,Shanghai
1,16,2022.378607,Hong Kong,Taiwan
2,960,2022.431357,Hong Kong,Guangxi
3,975,2022.466957,Hong Kong,Yunnan
4,33,2022.471557,Hong Kong,Taiwan
...,...,...,...,...
2095,2046,2023.794527,Henan,Gansu
2096,2049,2023.799997,Henan,Gansu
2097,2057,2023.802737,Henan,Gansu
2098,2078,2023.805477,Hong Kong,Gansu


In [15]:
a = set(df.Origin.tolist())
b = set(df.Destination.tolist())
all_province = a| b
len(set(all_province))

32

In [16]:
# https://www.qsl.net/bg4alc/ham/qthcn.htm
import re
# Python 度分秒 转 经纬度
# longitude = '41°8′32.01″'
def longitude_(longitude):
    longitude_split = re.split(u"°|\′|\″", longitude)[:2]
    if len(longitude_split) == 2:
        x = [float(j) for j in longitude_split]
        data = (x[0] + x[1] / 60 )
        return str('%.6f'% float(data))
# data = (longitude_(longitude))
# print(data)

In [3]:
df = pd.read_csv("/home/soniali/Desktop/02_china_recom_renew/2_treetime/2024-02-23_mugration/annotated_tree_events_del.csv",index_col = 0)
df

,EventTime,Origin,Destination
1103,2022.193397,Hong Kong,Shanghai
16,2022.378607,Hong Kong,Taiwan
960,2022.431357,Hong Kong,Guangxi
975,2022.466957,Hong Kong,Yunnan
33,2022.471557,Hong Kong,Taiwan
...,...,...,...
2046,2023.794527,Henan,Gansu
2049,2023.799997,Henan,Gansu
2057,2023.802737,Henan,Gansu
2078,2023.805477,Hong Kong,Gansu


In [4]:
a = set(df["Origin"].tolist())
b = set(df["Destination"].tolist())
all_province = set(a | b)

In [5]:
df_from_to  = pd.DataFrame(columns=list(all_province),index=list(all_province))
df_from_to = df_from_to.fillna(0)

In [6]:
for i in df.index:
    if df.loc[i,"Origin"] == "UNKNOWN" or df.loc[i,"Destination"] == "UNKNOWN" :
        continue
    else:
        loc_from = df.loc[i,"Origin"]
        loc_to = df.loc[i,"Destination"]
        df_from_to.loc[loc_from,loc_to] +=1

In [7]:
df_from_to = pd.DataFrame(df_from_to, dtype="int")

In [8]:
df_from_to.to_csv("/home/soniali/Desktop/02_china_recom_renew/2_treetime/china_3945_from_to province_all_clus.csv",sep = ",")

In [ ]:
# 运行 /home/soniali/Desktop/02_china_recom_renew/2_treetime/China_Dissemination_maps.R/China_Dissemination_maps.R

In [9]:
##从R中生成all_top50.csv文件
df_top50 = pd.read_csv("/home/soniali/Desktop/02_china_recom_renew/2_treetime/d_alpha_links_full_top50.csv")
df_top50
df_from_to_top50  = pd.DataFrame(columns=list(set(df_top50["Destination"].tolist())),index=list(set(df_top50["Origin"].tolist())))
df_from_to_top50 = df_from_to_top50.fillna(0)

for i in df_from_to_top50.index:
    for j in list(df_from_to_top50):
        df_0 = df_top50[df_top50["Origin"] == i]
        if j in df_0["Destination"].tolist():
            df_1 = df_0[df_0["Destination"] == j]
            num = int(df_1["n"])
            df_from_to_top50.loc[i,j]=num
        else:
            df_from_to_top50.loc[i,j]=0

df_from_to_top50.to_csv("/home/soniali/Desktop/02_china_recom_renew/2_treetime/china_3945_from_to province_top50_clus.csv",sep = ",")

/tmp/ipykernel_198222/2211757071.py:12: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  num = int(df_1["n"])


In [10]:
df_from_to_top50

,Sichuan,Jiangxi,Shanghai,Hong Kong,Beijing,Fujian,Anhui,Taiwan,Gansu,Jiangsu,Guangdong,Qinghai,Heilongjiang,Henan,Hubei,Xinjiang,Yunnan,Shanxi,Zhejiang
Sichuan,0,0,24,0,0,0,0,0,0,0,0,0,0,0,0,0,21,0,0
Henan,0,0,11,0,0,0,0,0,13,0,0,0,0,0,0,0,0,12,0
Hubei,31,26,22,0,18,19,20,0,18,20,16,11,22,26,0,23,27,17,22
Jiangxi,0,0,0,0,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0
Anhui,0,0,19,0,0,14,0,0,0,13,0,0,0,14,0,0,0,14,0
Taiwan,0,0,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Yunnan,0,0,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Jiangsu,0,0,25,0,0,0,23,0,0,0,17,0,14,18,13,0,16,0,13
Guangdong,0,0,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Shanghai,16,0,0,0,11,0,13,0,0,18,19,0,11,18,11,0,13,0,0


In [ ]:
# 先后运行China_Dissemination_maps.R和China_Dissemination_maps_3wave.R